We start by simulating a 1-D lattice (representing a string)

$F_{ij} = \frac{k\Delta z(r - r_0)}{r}$, $\Delta z = z_j - z_i$, $r = \sqrt{\Delta z^2 + d^2}$

$m_i \ddot z_i = -F_{(i-1)i} + F_{i(i+1)}$, $z_0 = 0$, $z_{N-1} = 0$

In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from manim import *
from dataclasses import dataclass

In [2]:
@dataclass
class Params():
    N: int = 25          # Number of masses
    m: float = 0.2      # Mass of each mass (kg)
    k: float = 10.     # Spring constant (N/m)
    r_0: float = 0.1    # Spring equilibrium distance (m)
    d: float = 0.5      # In-plane distance between adjacent masses (m)
    t_max: float = 10.0     # Total time to simulate (s)
    dt: float = 0.01    # Time step (s)

class String():
    
    """
    N = number of masses in the system
    m = mass of each mass
    r_0 = spring equilibrium distance
    d = in-plane distance between adjacent masses
    """
    def __init__(self, p: Params):
        self.N = p.N
        self.m = p.m
        self.k = p.k
        self.r_0 = p.r_0
        self.d = p.d
    
    def dy_dt(self, t, z_arr):

        z = z_arr[:self.N]
        z_dot = z_arr[self.N:]

        z_ddot = np.zeros(self.N)
        
        #Boundary conditions: fixed ends
        z_ddot[0] = 0
        z_ddot[-1] = 0

        for i in range(1, self.N - 1):
            #z_ddot[i] = (self.k / self.m) * (z[i+1] - 2 * z[i] + z[i-1])
            r_1 = np.sqrt(self.d**2 + (z[i+1] - z[i])**2)
            r_2 = np.sqrt(self.d**2 + (z[i] - z[i-1])**2)
            F_1 = self.k * (r_1 - self.r_0) * (z[i+1] - z[i]) / r_1
            F_2 = self.k * (r_2 - self.r_0) * (z[i] - z[i-1]) / r_2
            z_ddot[i] = (F_1 - F_2) / self.m

        return np.concatenate((z_dot, z_ddot))
    
    def solve_ode(self, t_pts, z_0, z_dot_0, 
                  abserr=1.0e-10, relerr=1.0e-10):
        """
        Solve the ODE given initial conditions.
        For now use odeint, but we have the option to switch.
        Specify smaller abserr and relerr to get more precision.
        """
        z_arr = np.concatenate((z_0, z_dot_0)) 
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             z_arr, t_eval=t_pts, 
                             atol=abserr, rtol=relerr)
        return solution.y

In [3]:
# Create String instance
params = Params()

def solve_system(params):
    string = String(params)

    t_pts = np.arange(0, params.t_max, params.dt)
    z_0 = np.zeros(params.N)
    z_dot_0 = np.zeros(params.N)
    z_0[params.N // 2] = 1  # Initial displacement at the center mass
    solution = string.solve_ode(t_pts, z_0, z_dot_0)
    return t_pts, solution

t_pts, sol = solve_system(params)
z = sol[:params.N, :]

# Plot results
"""
fig, ax = plt.subplots(params.N, 1, figsize=(10, 2*params.N))

for i in range(params.N):
    ax[i].plot(t_pts, z[i, :])
    ax[i].set_title(f'Mass {i+1} Displacement Over Time')
    ax[i].set_xlabel('Time')
    ax[i].set_ylabel('Displacement')
"""

"\nfig, ax = plt.subplots(params.N, 1, figsize=(10, 2*params.N))\n\nfor i in range(params.N):\n    ax[i].plot(t_pts, z[i, :])\n    ax[i].set_title(f'Mass {i+1} Displacement Over Time')\n    ax[i].set_xlabel('Time')\n    ax[i].set_ylabel('Displacement')\n"

In [4]:
# -------------------------------
# Simple spring helper (zig-zag)
# -------------------------------
def spring_polyline(start, end, coils=6, amplitude=0.25, inset=0.35):
    """
    Returns a VMobject shaped like a planar coil spring from start -> end.
    Uses set_points_as_corners for a crisp zig-zag. Compatible with manim v0.19.
    """
    start = np.array(start, dtype=float)
    end   = np.array(end, dtype=float)
    vec = end - start
    L = np.linalg.norm(vec)
    if L < 1e-6:
        return Line(start, end, stroke_width=6)

    # Local frame
    xhat = vec / L
    up = np.array([0.0, 1.0, 0.0])
    yhat = up - np.dot(up, xhat) * xhat
    ny = np.linalg.norm(yhat)
    if ny < 1e-8:
        right = np.array([1.0, 0.0, 0.0])
        yhat = right - np.dot(right, xhat) * xhat
        yhat /= np.linalg.norm(yhat)
    else:
        yhat /= ny

    # Straight end segments + zig-zag body
    Lz = max(L - 2 * inset, 0.0)
    n_verts = 2 * coils + 1
    xs = np.linspace(inset, inset + Lz, n_verts)

    ys = np.zeros_like(xs)
    ys[1::2] =  amplitude
    ys[2::2] = -amplitude
    # Ensure the last zig-zag point is on the center line
    if n_verts > 0:
        ys[-1] = 0

    pts = [start, start + xhat * inset]
    for xi, yi in zip(xs, ys):
        pts.append(start + xhat * xi + yhat * yi)
    pts += [end - xhat * inset, end]

    pts = np.array(pts, dtype=float)

    spring = VMobject()
    spring.set_points_as_corners(pts)
    spring.set_stroke(width=6)
    spring.set_fill(opacity=0)
    return spring

In [5]:
# make these "global" so that we can define various scenes with different initial conditions
FIG = 11.2
params = Params()

# -------------------------------
# Manim Scene
# -------------------------------
class String1D(Scene):
    def construct(self):
        # ---- Parameters ----
        # Physics
        fps_sample = 240    # samples per second for ODE solution and interpolation

        t_pts, sol = solve_system(params)
        z = sol[:params.N, :]

        m, k, r_0, d = params.m, params.k, params.r_0, params.d

        # Create the main title
        title = Text(f"1-D Spring Lattice", font_size=38).to_edge(UP)

        # Create the subtitle with spring constants
        subtitle = Text(
            f"m={m}, k={k}, r_0={r_0}, d={d}",
            font_size=32,
            color=WHITE
        ).next_to(title, DOWN)

        self.add(title, subtitle)

        # Interpolators for animation time -> displacement
        def z_of(t, i):
            return np.interp(t, t_pts, z[i])


        # Mass blocks (centered vertically, move along x only)
        x_ext = 6

        mass_dim = x_ext / 2 / params.N
        masses = np.zeros(params.N, dtype=object)
        x_eq = np.linspace(-x_ext, x_ext, params.N)
        for i in range(params.N):
            masses[i] = Rectangle(width=mass_dim, height=mass_dim, color=PINK, fill_opacity=1.0)
            masses[i].move_to([x_eq[i], 0.0, 0.0])
            self.add(masses[i])

        # Time tracker (drives the animation 1:1 with real time)
        t_tracker = ValueTracker(0.0)

        # Updaters for the masses (position vs. t)
        def mass_updater(mob, i):
            t = t_tracker.get_value()
            z = z_of(t, i)
            mob.move_to([x_eq[i], z, 0.0])


        for i in range(params.N):
            masses[i].add_updater(lambda mob, i=i: mass_updater(mob, i))

        # Use always_redraw so geometry refreshes as the masses move.
        springs = np.zeros(params.N - 1, dtype=object)

        # Create springs by binding the left/right mass objects at definition time
        # Compute endpoints from centers with small horizontal offsets to avoid relying on get_left/get_right
        for i in range(0, params.N - 1):
            num_coils = 5

            m_left = masses[i]
            m_right = masses[i+1]

            # Create a concrete spring VMobject and add a manual updater
            start = m_left.get_right()
            end = m_right.get_left()
            spr = spring_polyline(start, end, coils=num_coils, amplitude=mass_dim / 2, inset=mass_dim / 2).set_color(WHITE)

            def spr_updater(s, m_left=m_left, m_right=m_right):
                start = m_left.get_right()
                end = m_right.get_left()
                new = spring_polyline(start, end, coils=num_coils, amplitude=mass_dim / 2, inset=mass_dim / 2).set_color(WHITE)
                s.become(new)

            spr.add_updater(spr_updater)
            springs[i] = spr
            self.add(spr)

        # Optional: show a running time readout
        time_readout = DecimalNumber(
            number=0.0, num_decimal_places=2, include_sign=False
        ).set_font_size(28).to_corner(UR).shift(LEFT*1.1 + DOWN*1.5)
        time_label = Text("t (s) =", font_size=28).next_to(time_readout, LEFT, buff=0.2)

        def time_updater(mob):
            mob.set_value(t_tracker.get_value())

        time_readout.add_updater(time_updater)
        self.add(time_label, time_readout)

        # Animate: advance the tracker from 0 -> T_total in real time (rate_func=linear)
        self.play(t_tracker.animate.set_value(params.t_max), run_time=params.t_max, rate_func=linear)

        # Hold last frame briefly
        self.wait(0.5)


In [6]:
%manim -pql String1D

Manim Community v0.19.1

C:\Users\antho\AppData\Local\Temp\ipykernel_36996\63448221.py:89: DeprecationWarning: This method is not guaranteed to stay around. Please prefer setting the attribute normally or with Mobject.set().
  ).set_font_size(28).to_corner(UR).shift(LEFT*1.1 + DOWN*1.5)


[12/05/25 21:34:30] INFO     Animation 0 : Partial movie file written in                   ]8;id=536944;file://c:\Users\antho\anaconda3\envs\5300\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=174825;file://c:\Users\antho\anaconda3\envs\5300\Lib\site-packages\manim\scene\scene_file_writer.py#590\590]8;;\
                             'C:\Users\antho\Tony\gitclones\drumhead\media\videos\drumhead                         
                             \480p15\partial_movie_files\String1D\1584795214_2379264837_30                         
                             95388790.mp4'                                                                         

[12/05/25 21:34:31] INFO     Animation 1 : Partial movie file written in                   ]8;id=284017;file://c:\Users\antho\anaconda3\envs\5300\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=75757;file://c:\Users\antho\anaconda3\envs\5300\Lib\site-packages\manim\scene\scene_file_writer.py#590\590]8;;\
                             'C:\Users\antho\Tony\gitclones\drumhead\media\videos\drumhead                         
                             \480p15\partial_movie_files\String1D\4072820271_2574473371_28                         
                             29285616.mp4'                                                                         

                    INFO     Combining to Movie file.                                      ]8;id=785454;file://c:\Users\antho\anaconda3\envs\5300\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=536215;file://c:\Users\antho\anaconda3\envs\5300\Lib\site-packages\manim\scene\scene_file_writer.py#740\740]8;;\

                    INFO                                                                   ]8;id=461873;file://c:\Users\antho\anaconda3\envs\5300\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=552205;file://c:\Users\antho\anaconda3\envs\5300\Lib\site-packages\manim\scene\scene_file_writer.py#887\887]8;;\
                             File ready at                                                                         
                             'C:\Users\antho\Tony\gitclones\drumhead\media\videos\drumhead                         
                             \480p15\String1D.mp4'                                                                 
                                                                                                                   

                    INFO     Rendered String1D                                                         ]8;id=95367;file://c:\Users\antho\anaconda3\envs\5300\Lib\site-packages\manim\scene\scene.py\scene.py]8;;\:]8;id=76302;file://c:\Users\antho\anaconda3\envs\5300\Lib\site-packages\manim\scene\scene.py#279\279]8;;\
                             Played 2 animations                                                                   

                    INFO     Previewed File at:                                                     ]8;id=308425;file://c:\Users\antho\anaconda3\envs\5300\Lib\site-packages\manim\utils\file_ops.py\file_ops.py]8;;\:]8;id=604294;file://c:\Users\antho\anaconda3\envs\5300\Lib\site-packages\manim\utils\file_ops.py#237\237]8;;\
                             'C:\Users\antho\Tony\gitclones\drumhead\media\videos\drumhead\480p15\S                
                             tring1D.mp4'                                                                          